# Week 5: Embedding Database Optimization

This week’s assignment extends the previous weeks’ work by combining dense-vector semantic search with sparse keyword-based filtering. You will build a **hybrid retrieval system** that stores document chunks along with metadata in a SQLite+FAISS index, and performs both FAISS (vector) and full-text keyword search.  The idea is to “have the best of both worlds”: exact keyword matches (via SQLite FTS5 or BM25) **and** semantic similarity (via FAISS). In practice, hybrid methods (e.g. weighted score fusion or reciprocal rank fusion) can improve result relevance over using vectors or keywords alone.



## 📚 Learning Objectives

* **Hybrid Retrieval Techniques:** Understand how to combine dense vector search (FAISS) with sparse keyword search (SQLite FTS5 or BM25). Learn why pure semantic or pure keyword search alone can miss relevant results, and how hybrid search can address both broad “vibe” matches and exact queries.

* **Metadata and Indexing:** Learn to store document metadata (title, authors, year, keywords, etc.) alongside text chunks and their embeddings. You will design a combined index (SQLite tables + FAISS index) so that each text chunk has associated metadata fields.

* **Hybrid Ranking Strategies:** Explore ranking or fusion strategies to merge vector and keyword scores. For example, you might compute a weighted sum of normalized scores, or use **reciprocal rank fusion (RRF)** as a simple ensemble. The goal is to experiment with different score-combination methods and weightings for the final ranking.

* **Evaluation (Recall/Hit Rate):** Learn to evaluate retrieval quality. You should measure metrics like **recall** or **hit rate** (the proportion of queries where a relevant doc appears in the top-k results). You will compare the effectiveness of vector-only search, keyword-only search, and the hybrid approach on example queries.

## Project Design

You will **extend your Week 4 project** by adding metadata storage and keyword search, then implementing a hybrid retrieval pipeline. Key tasks include:

* **Index Structure:** Build or extend your SQLite + FAISS index to store *document metadata*, text *chunks*, and their *embeddings*. For example, use a SQLite table `documents(doc_id, title, author, year, keywords, ...)` and an FTS5 table (e.g. `doc_chunks`) that indexes the chunk text. The FAISS index should store the corresponding embeddings (one embedding per chunk) keyed by `doc_id`.

* **Keyword Search (FTS5/BM25):** Implement sparse keyword search over the text chunks. The simplest way is to use **SQLite FTS5**: create a virtual FTS table on the chunk text, so that queries like `WHERE doc_chunks MATCH 'term'` return relevant rows. Alternatively, you can use a BM25 library (e.g. Python’s `rank_bm25`) to rank chunks by BM25 similarity. Either way, you should be able to retrieve the top-k chunks by exact keyword relevance.

* **Hybrid Retrieval:** For a given user query, perform **two separate searches**: one in FAISS (semantic search) and one in the keyword index (FTS5 or BM25). Each returns its own top-k results with scores. You will then *merge* these results. For example, you could normalize the FAISS distances and FTS/BM25 scores, then compute a **weighted sum** or use **reciprocal rank fusion (RRF)** to re-rank the combined set.  The final output should be a single ranked list of document chunks or pages.

* **Performance Evaluation:** Design an evaluation to compare methods. For instance, prepare at least **10 test queries** with known relevant documents. Then measure for each method (vector-only, keyword-only, hybrid) how often a relevant document appears in the top-3 (or top-k) results — i.e., the hit rate or recall. Report these metrics (e.g. “Recall\@3”) to see whether hybrid search improves over the baselines.


Create virtual environment

(base) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>conda create -n mod5venv python=3.10.18

Activate virtual environment

(base) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>conda activate mod5venv

Install Pytorch

(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

Install faiss-gpu

(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>conda install -c pytorch faiss-gpu

pysqlite3 is a Python wrapper around the native SQLite C library.
To compile it, you need:
The SQLite development headers (sqlite3.h)
A compiled version of libsqlite3 (or ability to link against it)
On Windows, these are not available by default, and the build tools (like MSVC) can't locate them.
Even though your system has SQLite (Python includes a built-in sqlite3 module), it uses the embedded version and does not expose the development headers needed to build pysqlite3.

The easiest and most reliable fix is to avoid compiling pysqlite3 altogether by using the pre-compiled binary wheel:
pip install pysqlite3-binary
✅ This package:
Contains the same API as pysqlite3
Includes a bundled SQLite3 library
Does not require compilation
Works out of the box on Windows, macOS, and Linux

Despite being a popular package, pysqlite3-binary has no official wheels for some platforms, including: Python 3.10+ on Windows (AMD64) — sometimes missing from PyPI pip install pysqlite3-binary so the above command does not work.Insted, we have to use:

(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>conda install -c conda-forge pysqlite3

Install Other Required Packages

BM25 (Best Matching 25) is a ranking function used in information retrieval to estimate the relevance of documents to a search query. It's an improvement over the classic TF-IDF method and is widely used in search engines and question-answering systems.

BM25 scores documents based on:

Term frequency (how often a query word appears in a document)
Inverse document frequency (how rare the word is across all documents)
Document length (longer documents are normalized to avoid bias)

rank-bm25 is a Python library that implements the BM25 algorithm. It allows you to rank a list of documents based on how relevant they are to a given query.

It supports:

BM25Okapi: the standard version
BM25L and BM25Plus: variants with improved scoring in certain cases

jupyter is included to install the Jupyter Notebook interface, which is a popular tool for writing and running Python code interactively.


FastAPI is a high-performance framework for building RESTful APIs. It’s designed to be fast, easy to use, and fully compatible with Python type hints.

(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>pip install openai transformers sentence-transformers rank_bm25 python-dotenv jupyter fastapi uvicorn

Sentence_transformers may be installed for the second time if it can not be verified.

Sentence_transformer is a Python library built on top of Hugging Face's transformers and PyTorch. It makes it easy to generate sentence embeddings using pre-trained models like BERT, RoBERTa, etc., with minimal code.


(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>pip install sentence_transformers

More packages need to be installed or reinstalled

(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>pip install python-dotenv

(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>pip install openai

(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>pip install langchain

(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>conda install -c conda-forge ipywidgets

(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>pip install fastapi uvicorn

Verify GPU & Environment Setup

In [1]:
import torch
import faiss
import sys
from sentence_transformers import SentenceTransformer

print("✅ Python Version:", sys.version)
print("✅ PyTorch Version:", torch.__version__)
print("✅ CUDA Available:", torch.cuda.is_available())
print("✅ GPU Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
print("✅ FAISS GPU Support:", hasattr(faiss, 'GpuIndexFlat'))

# Test embedding model
print("✅ Loading test model...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Model loaded successfully.")

✅ Python Version: 3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:42:04) [MSC v.1943 64 bit (AMD64)]
✅ PyTorch Version: 2.5.1
✅ CUDA Available: True
✅ GPU Device: NVIDIA GeForce RTX 4070 SUPER
✅ FAISS GPU Support: True
✅ Loading test model...
✅ Model loaded successfully.


Use .env for api keys

In [2]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

Project Structure

Week5/
├── mod5venv/                  # Virtual environment
├── data/
│   └── documents.jsonl         # Your test documents
├── db/
│   └── mydata.db               # SQLite + FTS5 + FAISS
├── embeddings/
│   └── faiss_index.bin         # FAISS index
├── evaluate.ipynb              # Evaluation notebook
├── app.py                      # FastAPI endpoint
└── .env                        # API keys

Load and Chunk Documents

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Read text from a file
with open("langchain.txt", "r", encoding="utf-8") as file: # langchain is the text file with 2,320 characters
    document_text = file.read()

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)
chunks = splitter.split_text(document_text)

print(f"Number of chunks: {len(chunks)}")
print(f"First chunk: {chunks[0]}")
print(f"second chunk: {chunks[1]}")
print(f"last chunk: {chunks[6]}")

Number of chunks: 7
First chunk: LangChain is an open source framework for building applications based on large language models (LLMs). LLMs are large deep-learning models pre-trained on large amounts of data that can generate responses to user queries—for example, answering questions or creating images from text-based prompts. LangChain provides tools and abstractions to improve the customization, accuracy, and relevancy of the information the models generate. For example, developers can use LangChain components to build new prompt chains
second chunk: can use LangChain components to build new prompt chains or customize existing templates. LangChain also includes components that allow LLMs to access new data sets without retraining.
last chunk: and asynchronous subprocess runs; and the Wolfram Alpha website and SDK. As of April 2023, it can read from more than 50 document types and data sources.


So even though 2320 / 512 ≈ 4.5, you get 7 chunks because:

* Overlap adds redundancy
* Early splits due to text structure reduce average chunk size
* Final chunks may be small

## Generate Embeddings ##

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight, runs locally
embeddings = model.encode(chunks)
print(f"Generated {len(embeddings)} embeddings.")
print(embeddings.shape)  # Output: (3, 384)
print(len(embeddings[0]))     # First vector: array of 384 floats

Generated 7 embeddings.
(7, 384)
384


SentenceTransformer is the main class in the sentence-transformers library that you use to load and work with models.

all-MiniLM-L6-v2 is the name of a pre-trained model specifically designed for generating sentence embeddings.

Even though your text chunks have different lengths (number of characters or words), each one still gets a fixed-size embedding vector of 384 dimensions.

## Set Up SQLite + FTS5 ##

restart the Kernel before run the below cell.

In [6]:
# restartt the Kernel before run this cell.

import sqlite3
import os


# --- 🔥 DANGER ZONE: Uncomment to DELETE existing DB (for clean dev runs)
if os.path.exists("db/mydata.db"):
    os.remove("db/mydata.db")
conn = sqlite3.connect("db/mydata.db")
# --- 🔥 DANGER ZONE END ---

# Create the directory if it doesn't exist
os.makedirs("db", exist_ok=True)

conn = sqlite3.connect("db/mydata.db")

# Example values - replace these with real data or variables from your code
# doc_id = 1 # or use a UUID, or autoincremental etc.
title = "Sample Research Paper"
author = "John Doe"
year = 2025
keywords = "machine learning, NLP, AI"
# chunk_id = 1
chunk_text = "This is a sample chunk of text from the document."

# Now execute your SQL statements

# Metadata table
conn.execute("""
    CREATE TABLE IF NOT EXISTS documents (
        doc_id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        author TEXT,
        year INTEGER,
        keywords TEXT
    )
""")

# FTS5 Table for keyword search
# FTS: Full-Text Search table  
# content: This is the content of the chunk. Indexed for search. 
# content_rowid: the row ID of the chunk, not indexed for search.
# doc_id: Foreign key to documents table. Not indexed for search.
conn.execute("""
    CREATE VIRTUAL TABLE IF NOT EXISTS doc_chunks USING fts5( 
        content, 
        content_rowid UNINDEXED,  
        doc_id UNINDEXED
    )
""")

# Insert a document into the metadata table
# Use parameterized queries to prevent SQL injection
conn.execute("""INSERT INTO documents (title, author, year, keywords) VALUES (?, ?, ?, ?)""",
             (title, author, year, keywords))


# Get the auto-generated ID
# cursor = conn.cursor()
# doc_id = cursor.lastrowid
# ✅ CORRECT: Get last inserted ID using SQL function
doc_id = conn.execute("SELECT last_insert_rowid()").fetchone()[0]
print(f"Inserted document with auto doc_id = {doc_id}")

# Insert chunk for FTS5
chunk_id = 1  # This should be unique for each chunk, could be an auto-increment or UUID
conn.execute("""INSERT INTO doc_chunks(rowid, content, doc_id) VALUES (?, ?, ?)""",
             (chunk_id, chunk_text, doc_id))

conn.commit()

# Debug: Check what's in the table --  this is just for verification
rows = conn.execute("SELECT doc_id, title FROM documents").fetchall()
print("All documents:", rows)

conn.close()

print("✅ Database setup complete.")

Inserted document with auto doc_id = 1
All documents: [(1, 'Sample Research Paper')]
✅ Database setup complete.


Inspect the doc -chunk table

In [8]:
import sqlite3
import os

# Connect to DB
conn = sqlite3.connect("db/mydata.db")

# === Check if table exists ===
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='doc_chunks';").fetchall()
if not tables:
    print("❌ Table 'doc_chunks' does not exist.")
else:
    print("✅ Table 'doc_chunks' exists.")

    # === Show schema ===
    print("\n📋 Table Schema:")
    schema = conn.execute("PRAGMA table_info(doc_chunks);").fetchall()
    for col in schema:
        print(f"  {col}")

    # === Show all data ===
    print("\n📦 Contents of doc_chunks:")
    rows = conn.execute("SELECT rowid, content, doc_id FROM doc_chunks").fetchall()
    if rows:
        for row in rows:
            print(f"  Row {row[0]}: doc_id={row[2]} → '{row[1]}'")
    else:
        print("  (No data found)")

    # === Full-text search example ===
    print("\n🔍 Full-text search for 'sample':")
    results = conn.execute("""
        SELECT rowid, content, doc_id 
        FROM doc_chunks 
        WHERE content MATCH 'sample'
    """).fetchall()
    for r in results:
        print(f"  Matched: {r}")

conn.close()

✅ Table 'doc_chunks' exists.

📋 Table Schema:
  (0, 'content', '', 0, None, 0)
  (1, 'content_rowid', '', 0, None, 0)
  (2, 'doc_id', '', 0, None, 0)

📦 Contents of doc_chunks:
  Row 1: doc_id=1 → 'This is a sample chunk of text from the document.'

🔍 Full-text search for 'sample':
  Matched: (1, 'This is a sample chunk of text from the document.', 1)


## Build FAISS Index ##

Embeddings are from previous cell

In [11]:
import faiss
import numpy as np
import os

# Ensure the directory exists
os.makedirs("embeddings", exist_ok=True) # Create directory if it doesn't exist

# Assume 'embeddings' is already defined and is a numpy array of shape (n, d)
dimension = embeddings.shape[1] # Get the dimension of the embeddings
print(f"Embedding dimension: {dimension}")

# Create GPU resources
res = faiss.StandardGpuResources() # Create resources for GPU index

# Build GPU index
index_gpu = faiss.GpuIndexFlatL2(res, dimension) 
# Note: If you want to use CPU, you can use IndexFlatL2 for CPU

# Add embeddings
index_gpu.add(np.array(embeddings))

# ====> Step 1: Convert GPU index to CPU index <====
index_cpu = faiss.index_gpu_to_cpu(index_gpu)

# ====> Step 2: Save the CPU index
faiss.write_index(index_cpu, "embeddings/faiss_index.bin")

print("✅ FAISS index saved successfully.")

Embedding dimension: 384
✅ FAISS index saved successfully.


To Load Later (Back to GPU if Needed):

In [ ]:
# Load CPU index
index_cpu = faiss.read_index("embeddings/faiss_index.bin")

# Move index back to GPU (if needed)
res = faiss.StandardGpuResources()
index_gpu = faiss.index_cpu_to_gpu(res, 0, index_cpu)  # device 0

## Hybrid Search  ##

In [ ]:
def hybrid_search(query, k=3, alpha=0.6): # Hybrid Search Pipeline k=3, alpha=0.6   
    # 1. Vector Search (FAISS)  
    query_vec = model.encode([query]) # Convert query to embedding
    D, I = index.search(np.array(query_vec), k) # Search the index
    faiss_results = list(zip(I[0], 1 / (1 + D[0])))  # Convert distance to score

    # 2. Keyword Search (FTS5)
    cursor = conn.cursor() # Use the existing connection
    # Use parameterized query to prevent SQL injection
    query = f"{query}*"  # FTS5 wildcard search
    # Fetch top-k results based on keyword match
    cursor.execute("""
        SELECT doc_id, rank FROM doc_chunks
        WHERE content MATCH ?
        ORDER BY rank
        LIMIT ?""", (query, k)) 
    keyword_results = [(row[0], row[1]) for row in cursor.fetchall()] # 

    # Normalize scores (min-max or rank-based)
    def normalize_scores(results): 
        if not results: return {}
        scores = {r[0]: r[1] for r in results}
        min_s, max_s = min(scores.values()), max(scores.values())
        if max_s == min_s:
            return {doc: 0.5 for doc in scores}
        return {doc: (s - min_s) / (max_s - min_s) for doc, s in scores.items()}

    v_scores = normalize_scores(faiss_results)
    k_scores = normalize_scores(keyword_results)

    # 3. Weighted Fusion
    combined = {}
    for doc_id in set(v_scores.keys()) | set(k_scores.keys()): 
        v = v_scores.get(doc_id, 0.0)
        k = k_scores.get(doc_id, 0.0)
        combined[doc_id] = alpha * v + (1 - alpha) * k

    # Sort and return top-k
    sorted_docs = sorted(combined.items(), key=lambda x: x[1], reverse=True) 
    return sorted_docs[:k]

FastAPI Endpoint

In [ ]:
!pip show fastapi

Name: fastapi
Version: 0.116.1
Summary: FastAPI framework, high performance, easy to learn, fast to code, ready for production
Home-page: https://github.com/fastapi/fastapi
Author: 
Author-email: =?utf-8?q?Sebasti=C3=A1n_Ram=C3=ADrez?= <tiangolo@gmail.com>
License: 
Location: c:\users\ch939\anaconda3\envs\mod5venv\lib\site-packages
Requires: pydantic, starlette, typing-extensions
Required-by: 


Run the below command in Anaconda prompt instead.

(mod5venv) C:\Users\ch939\Downloads\LLMBootCampCodes\Week5>python main.py
INFO:     Started server process [14332]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     127.0.0.1:60387 - "GET /hybrid_search?query=natural+language+processing HTTP/1.1" 200 OK
INFO:     127.0.0.1:60387 - "GET /favicon.ico HTTP/1.1" 404 Not Found

In [ ]:
# Do not run the following code in Jupyter Notebook, run it in a separate Python file or terminal.
# This is a FastAPI application to expose the hybrid search functionality as an API.

from fastapi import FastAPI
import uvicorn

app = FastAPI()

@app.get("/hybrid_search") # Hybrid Search API endpoint
async def hybrid_search_api(query: str, k: int = 3): # 
    results = hybrid_search(query, k)
    return {"results": [{"doc_id": r[0], "score": r[1]} for r in results]}

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)


## Starter Code Snippets

Here are some example code snippets and schemas to help you get started:

* **SQLite schema:** Define a table for document metadata and an FTS5 table for text. For example:

  ```sql
  CREATE TABLE documents (
      doc_id    INTEGER PRIMARY KEY,
      title     TEXT,
      author    TEXT,
      year      INTEGER,
      keywords  TEXT
  );
  CREATE VIRTUAL TABLE doc_chunks USING fts5(
      content,                      -- chunk text
      content='documents',          -- external content table
      content_rowid='doc_id'        -- link to documents.doc_id
  );
  ```

  This creates an FTS5 index on the `content` column (chunk text) referencing the `documents` table.

* **Inserting data:** Insert your documents and chunk text. For example, in Python:

  ```python
  conn = sqlite3.connect("mydata.db")
  # Insert document metadata
  conn.execute("INSERT INTO documents VALUES (?, ?, ?, ?, ?)",
               (doc_id, title, author, year, keywords))
  # Insert chunk text into FTS table, linking by rowid
  conn.execute("INSERT INTO doc_chunks(rowid, content) VALUES (?, ?)",
               (doc_id, chunk_text))
  conn.commit()
  ```

  Or in raw SQL, you might SELECT from a content table into the FTS table as shown in.

* **Keyword query (FTS5):** A full-text query can be written as:

  ```sql
  SELECT doc_id, title
  FROM documents
  JOIN doc_chunks ON documents.doc_id = doc_chunks.rowid
  WHERE doc_chunks MATCH 'search terms'
  LIMIT 5;
  ```

  This returns documents whose chunks match the query terms.

* **BM25 example (Python):** If you use `rank_bm25`, example usage is:

  ```python
  from rank_bm25 import BM25Okapi
  docs = ["text of doc1 ...", "text of doc2 ...", ...]
  tokenized_docs = [doc.split() for doc in docs]
  bm25 = BM25Okapi(tokenized_docs)
  query = "example query"
  tokenized_query = query.split()
  top_docs = bm25.get_top_n(tokenized_query, docs, n=3)
  ```

  This returns the top 3 documents by BM25 score.

* **Hybrid score merging:** Here’s a simple Python example of a weighted-sum merger:

  ```python
  def hybrid_score(vec_score, key_score, alpha=0.5):
      # Assume vec_score and key_score are normalized (0-1).
      return alpha * vec_score + (1 - alpha) * key_score

  # Example usage for re-ranking top results:
  combined = []
  for doc, v_score in faiss_results:
      k_score = keyword_scores.get(doc, 0.0)
      combined_score = hybrid_score(v_score, k_score, alpha=0.6)
      combined.append((doc, combined_score))
  combined.sort(key=lambda x: x[1], reverse=True)
  top_k = combined[:3]
  ```

  You can adjust `alpha` or use other formulas (e.g. max, reciprocal rank fusion).

* **FastAPI route skeleton:** To serve hybrid search via an API, you might write:

  ```python
  from fastapi import FastAPI
  app = FastAPI()

  @app.get("/hybrid_search")
  async def hybrid_search(query: str, k: int = 3):
      # 1. Compute query embedding for FAISS
      # 2. Get top-k from FAISS and top-k from SQLite FTS/BM25
      # 3. Merge scores (as above) and select final top-k documents
      return {"results": top_k_results}
  ```

  This endpoint takes a `query` string and returns the top-k hybrid results in JSON.



## Deliverables

Your submission should include:

* The **updated SQLite+FAISS index** (or code to build it) that contains the document chunks, embeddings, and metadata.
* The **hybrid retrieval pipeline code**, including FAISS search, FTS/BM25 search, and the score-merging logic.
* An **evaluation notebook** (e.g. Jupyter) showing at least 10 example queries and reporting metrics (e.g. recall or hit rate @3) for vector-only, keyword-only, and hybrid search.
* A **FastAPI endpoint** implementation (`/hybrid_search`) that returns the hybrid top-3 results for a given query (as JSON).

Include comments in your code to explain each step. Your evaluation should show whether the hybrid method improves over using vectors or keywords alone.


**References:** This assignment is based on standard practices for hybrid search using FAISS and SQLite FTS5. Reciprocal rank fusion and other fusion methods are known techniques in information retrieval. For more background on hit rate metrics, see analytics tutorials on search evaluation.
